In [30]:
import pandas as pd

# Pfad zum Raw CSV-Datensatz (Liga-Tabelle)
league_path = '../data/df_league_table_raw.csv'

# Liga-Datensatz laden
league_df = pd.read_csv(league_path)

# Überprüfe die Spaltennamen – es sollten folgende vorhanden sein:
# Season, Spieltag, Rank, Team, Spiele, G, U, V, Tore, Goal_Diff, Points
print("Spalten im Raw DataFrame:", league_df.columns.tolist())

# Falls die Spalte "Rank" als numerischer Wert vorliegt, konvertieren wir sie ggf. in int:
league_df['Rank'] = pd.to_numeric(league_df['Rank'], errors='coerce')

# Erstelle die Zielvariable "relegated":
# Ein Verein steigt ab, wenn in der Spalte "Rank" der Wert 12 steht.
league_df['relegated'] = league_df['Rank'].apply(lambda x: 1 if x == 12 else 0)

# Zur Kontrolle: Zeige die ersten Zeilen des bereinigten DataFrames
league_df.head(15)


Spalten im Raw DataFrame: ['Season', 'Spieltag', 'Rank', 'Team', 'Spiele', 'G', 'U', 'V', 'Tore', 'Goal_Diff', 'Points']


,Season,Spieltag,Rank,Team,Spiele,G,U,V,Tore,Goal_Diff,Points,relegated
0,2024,1,1,FC Zürich,1,1,0,0,2:0,2,3,0
1,2024,1,2,Lausanne-Sport,1,1,0,0,3:2,1,3,0
2,2024,1,3,FC Lugano,1,1,0,0,2:1,1,3,0
3,2024,1,3,Servette FC,1,1,0,0,2:1,1,3,0
4,2024,1,3,FC Sion,1,1,0,0,2:1,1,3,0
5,2024,1,6,FC Winterthur,1,1,0,0,1:0,1,3,0
6,2024,1,7,FC Basel,1,0,0,1,2:3,-1,0,0
7,2024,1,8,BSC Young Boys,1,0,0,1,1:2,-1,0,0
8,2024,1,8,FC Luzern,1,0,0,1,1:2,-1,0,0
9,2024,1,8,Grasshoppers,1,0,0,1,1:2,-1,0,0


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Verwende das bereinigte DataFrame league_df aus Abschnitt 1

# Wähle die Features: 
# Wir verwenden hier "Points", "Goal_Diff", "G", "U" und "V".
features = ['Points', 'Goal_Diff', 'G', 'U', 'V']
X = league_df[features]
y = league_df['relegated']

# Aufteilen in Trainings- und Testdaten (80 % Training, 20 % Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skalierung der Features – wichtig z. B. für den SVC
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definiere drei ML-Modelle
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Support Vector Classifier': SVC(probability=True)
}

# Trainiere und evaluiere jedes Modell
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"Modell: {name}")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("-" * 50)

# Beispielhaft: Wähle hier das Random Forest-Modell als bestes Modell aus
best_model = models['Random Forest']

# Speichere das beste Modell und den Skalierer zur späteren Verwendung (z. B. in einem Flask-Service)
joblib.dump(best_model, 'best_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("Bestes Modell und Skalierer wurden gespeichert.")


Modell: Logistic Regression
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       114
           1       1.00      0.67      0.80         6

    accuracy                           0.98       120
   macro avg       0.99      0.83      0.90       120
weighted avg       0.98      0.98      0.98       120

Accuracy: 0.9833333333333333
--------------------------------------------------
Modell: Random Forest
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       114
           1       1.00      0.67      0.80         6

    accuracy                           0.98       120
   macro avg       0.99      0.83      0.90       120
weighted avg       0.98      0.98      0.98       120

Accuracy: 0.9833333333333333
--------------------------------------------------
Modell: Support Vector Classifier
              precision    recall  f1-score   support

           0       0.97      1.00      0.99    